# Homework 3: PCA/Hyperparameter/CV
Data source: http://archive.ics.uci.edu/ml/datasets/Polish+companies+bankruptcy+data

Name:Fang Yuting  
StudentID:1901212576  
Major:19Fintech

### pre-precessing
(loading to dataframe, creating bankruptcy column, changing column names, filling-in na values, training-vs-test split, standardizatino, etc)

In [1]:
from scipy.io import arff
import pandas as pd
import numpy as np

data = arff.loadarff('4year.arff')
df = pd.DataFrame(data[0])
df.head()

Attr1    Attr2    Attr3   Attr4    Attr5     Attr6     Attr7    Attr8  \
0  0.159290  0.46240  0.07773  1.1683  -44.853  0.467020  0.189480  0.82895   
1 -0.127430  0.46243  0.26917  1.7517    7.597  0.000925 -0.127430  1.16250   
2  0.070488  0.23570  0.52781  3.2393  125.680  0.163670  0.086895  2.87180   
3  0.136760  0.40538  0.31543  1.8705   19.115  0.504970  0.136760  1.45390   
4 -0.110080  0.69793  0.18878  1.2713  -15.344  0.000000 -0.110080  0.43282   

    Attr9   Attr10  ...    Attr56   Attr57   Attr58    Attr59   Attr60  \
0  1.1223  0.38330  ...  0.108990  0.41557  0.89101  0.001422   7.7928   
1  1.2944  0.53757  ... -0.089372 -0.23704  1.06250  0.150410   5.4327   
2  1.0574  0.67689  ...  0.054286  0.10413  0.94571  0.000000   7.1070   
3  1.1144  0.58938  ...  0.102630  0.23203  0.89737  0.073024   6.1384   
4  1.7350  0.30207  ...  0.439880 -0.36440  0.57153  0.000000  18.8010   

   Attr61   Attr62  Attr63   Attr64  class  
0  4.9914  119.810  3.0465   3.0560   b'0'  
1  3.4629  100.970  3.6150   3.4725   b'0'  
2  3.3808   76.076  4.7978   4.7818   b'0'  
3  4.2241   88.299  4.1337   4.6484   b'0'  
4  2.7925  146.390  2.4934  15.0360   b'0'  

[5 rows x 65 columns]

In [2]:
df['bankruptcy'] = (df['class']==b'1')
del df['class']
df.columns = ['X{0:02d}'.format(k) for k in range(1,65)] + ['bankruptcy']

df.describe()

X01          X02          X03           X04           X05  \
count  9791.000000  9791.000000  9791.000000   9749.000000  9.771000e+03   
mean      0.043019     0.596404     0.130959      8.136600  6.465164e+01   
std       0.359321     4.587122     4.559074    290.647281  1.475939e+04   
min     -12.458000     0.000000  -445.910000     -0.045319 -3.794600e+05   
25%       0.001321     0.263145     0.020377      1.047000 -5.121700e+01   
50%       0.041364     0.467740     0.199290      1.591800 -5.557600e-02   
75%       0.111130     0.689255     0.410670      2.880400  5.573200e+01   
max      20.482000   446.910000    22.769000  27146.000000  1.034100e+06   

               X06          X07           X08          X09          X10  ...  \
count  9791.000000  9791.000000   9773.000000  9792.000000  9791.000000  ...   
mean     -0.059273     0.059446     19.884016     1.882296     0.389040  ...   
std       6.812754     0.533344    698.697015    17.674650     4.590299  ...   
min    -486.820000   -12.458000     -1.848200    -0.032371  -445.910000  ...   
25%      -0.000578     0.003004      0.428300     1.006675     0.294440  ...   
50%       0.000000     0.048820      1.088700     1.161300     0.510450  ...   
75%       0.065322     0.126940      2.691000     1.970225     0.714290  ...   
max     322.200000    38.618000  53209.000000  1704.800000    12.602000  ...   

                X55          X56          X57          X58          X59  \
count  9.792000e+03  9771.000000  9791.000000  9776.000000  9791.000000   
mean   7.686330e+03    -0.992263     0.035022     1.133287     0.856053   
std    7.605261e+04    77.007971     8.945365     8.038201    26.393305   
min   -7.132200e+05 -7522.100000  -597.420000   -30.892000  -284.380000   
25%    2.184000e+01     0.003121     0.008768     0.885722     0.000000   
50%    9.503300e+02     0.043679     0.098026     0.958305     0.002129   
75%    4.694550e+03     0.117170     0.242680     0.996163     0.211790   
max    6.123700e+06   112.020000   226.760000   668.750000  1661.000000   

                 X60            X61           X62          X63           X64  
count    9178.000000    9760.000000  9.771000e+03  9749.000000   9561.000000  
mean      118.156064      25.194430  2.015157e+03     8.660813     35.949619  
std      3230.316692    1099.260821  1.171461e+05    60.838202    483.318623  
min         0.000000     -12.656000 -1.496500e+04    -0.024390     -0.000015  
25%         5.356325       4.267700  4.323400e+01     2.938800      2.012900  
50%         9.482000       6.283550  7.472900e+01     4.848900      4.041600  
75%        19.506000       9.938200  1.233450e+02     8.363800      9.413500  
max    251570.000000  108000.000000  1.077900e+07  5662.400000  21153.000000  

[8 rows x 64 columns]

In [3]:
sum(df.bankruptcy == True)

515

In [4]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
X_imp = imp_mean.fit_transform(df.values)

from sklearn.model_selection import train_test_split
X, y = X_imp[:, :-1], X_imp[:, -1]
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.3, 
                                                   random_state=0, stratify=y)

from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)

### Extract 3 features using PCA method.

In [6]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 3)
X_train_pca = pca.fit_transform(X_train_std)
X_test_pca = pca.transform(X_test_std)
print(X_train_pca.shape)
print(X_test_pca.shape)

(6854, 3)
(2938, 3)


### Using the selected features from above, we are going to apply LR / SVM / decision tree.

#### 1. Logistic Regression

In [9]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train_pca,y_train)
print('Training accuracy:', lr.score(X_train_pca, y_train))
print('Test accuracy:', lr.score(X_test_pca, y_test))

Training accuracy: 0.946892325649256
Test accuracy: 0.9462219196732471


#### 2.SVM

In [10]:
from sklearn.svm import SVC
svm = SVC(kernel = 'rbf', random_state = 1,gamma = 0.01, C = 1.0)
svm.fit(X_train_pca, y_train)
print('Training accuracy:', svm.score(X_train_pca, y_train))
print('Test accuracy:', svm.score(X_test_pca, y_test))

Training accuracy: 0.9476218266705574
Test accuracy: 0.9472430224642614


#### 3.Decision Tree

In [11]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(criterion = 'gini', max_depth = 4, random_state = 1)
tree.fit(X_train_pca, y_train)
print('Training accuracy:', tree.score(X_train_pca, y_train))
print('Test accuracy:', tree.score(X_test_pca, y_test))

Training accuracy: 0.948497227896119
Test accuracy: 0.9472430224642614


### Implement the methods using pipeline.

#### 1.Logistic Regression

In [12]:
from sklearn.pipeline import make_pipeline
pipe_lr = make_pipeline(StandardScaler(),
                        PCA(n_components = 3),
                        LogisticRegression())
pipe_lr.fit(X_train,y_train)
print('Training accuracy:', pipe_lr.score(X_train, y_train))
print('Test accuracy:', pipe_lr.score(X_test, y_test))

Training accuracy: 0.946892325649256
Test accuracy: 0.9462219196732471


#### 2.SVM

In [15]:
from sklearn.pipeline import make_pipeline
pipe_svm = make_pipeline(StandardScaler(),
                        PCA(n_components = 3),
                        SVC(kernel = 'rbf', random_state = 1,gamma = 0.01, C = 1.0))
pipe_svm.fit(X_train,y_train)
print('Training accuracy:', pipe_svm.score(X_train, y_train))
print('Test accuracy:', pipe_svm.score(X_test, y_test))

Training accuracy: 0.9476218266705574
Test accuracy: 0.9472430224642614


#### 3.Decision Tree

In [16]:
from sklearn.pipeline import make_pipeline
pipe_tree = make_pipeline(StandardScaler(),
                        PCA(n_components = 3),
                        DecisionTreeClassifier(criterion = 'gini', max_depth = 4, random_state = 1))
pipe_tree.fit(X_train,y_train)
print('Training accuracy:', pipe_tree.score(X_train, y_train))
print('Test accuracy:', pipe_tree.score(X_test, y_test))

Training accuracy: 0.948497227896119
Test accuracy: 0.9472430224642614


### Use grid search for finding optimal hyperparameters. 

#### 1.Logistic Regression

In [20]:
from sklearn.model_selection import GridSearchCV
param_grid = [{'logisticregression__C':[0.00001,0.0001,0.001,0.01,0.1,1,10]}]
gslr = GridSearchCV(estimator = pipe_lr,
                  param_grid= param_grid,
                  scoring='accuracy',
                  cv=10,n_jobs = -1)
gslr = gslr.fit(X_train, y_train)
print(gslr.best_score_) #best_score_ is f1 score
print(gslr.best_params_)
#After we do grid search, we find the best parameter. 
#Then, the best params model will fit automatically.
print('Train accuracy: ' ,str(gslr.score(X_train, y_train)))
print('Test accuracy: ' ,str(gslr.score(X_test, y_test)))

0.9474758996403567
{'logisticregression__C': 1e-05}
Train accuracy:  0.9474759264662971
Test accuracy:  0.9469026548672567


#### 2.SVM

In [21]:
param_range = [0.0001, 0.001,0.01, 0.1,1]
param_grid = [{'svc__C': param_range, 
               'svc__kernel': ['linear']},
              {'svc__C': param_range, 
               'svc__gamma': param_range, 
               'svc__kernel': ['rbf']}]
gssvm = GridSearchCV(estimator=pipe_svm, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  cv=10,
                  n_jobs=-1)
gssvm = gssvm.fit(X_train, y_train)
print(gssvm.best_score_)
print(gssvm.best_params_)
print('Train accuracy: ' ,str(gssvm.score(X_train, y_train)))
print('Test accuracy: ' ,str(gssvm.score(X_test, y_test)))

0.9474758996403567
{'svc__C': 0.0001, 'svc__kernel': 'linear'}
Train accuracy:  0.9474759264662971
Test accuracy:  0.9472430224642614


#### 3.Decision Tree

In [18]:
param_grid = [{'decisiontreeclassifier__max_depth':[1, 2, 3, 4, 5, 6, 7, None]}]
gstree = GridSearchCV(estimator = pipe_tree,
                  param_grid= param_grid,
                  scoring='accuracy',
                  cv=10,n_jobs = -1)
gstree = gstree.fit(X_train, y_train)
print(gstree.best_score_)
print(gstree.best_params_)
print('Train accuracy: ' ,str(gstree.score(X_train, y_train)))
print('Test accuracy: ' ,str(gstree.score(X_test, y_test)))

0.9474758996403567
{'decisiontreeclassifier__max_depth': 1}
Train accuracy:  0.9474759264662971
Test accuracy:  0.9472430224642614
